In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Input, Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

df_train = pd.read_csv('train.txt')
df_val = pd.read_csv('val.txt')
df_test = pd.read_csv('test.txt')


# train
df_train['X_real'] = df_train['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_train['X_img'] = df_train['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_train['y_real'] = df_train['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_train['y_img'] = df_train['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)

# test
df_test['X_real'] = df_test['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_test['X_img'] = df_test['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_test['y_real'] = df_test['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_test['y_img'] = df_test['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)

# val
df_val['X_real'] = df_val['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_val['X_img'] = df_val['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_val['y_real'] = df_val['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_val['y_img'] = df_val['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)


X_train = torch.tensor(df_train[['X_real','X_img']].to_numpy().tolist())
y_train = torch.tensor(df_train[['y_real','y_img']].to_numpy().tolist())

X_test = torch.tensor(df_test[['X_real','X_img']].to_numpy().tolist())
y_test = torch.tensor(df_test[['y_real','y_img']].to_numpy().tolist())

X_val = torch.tensor(df_val[['X_real','X_img']].to_numpy().tolist())
y_val = torch.tensor(df_val[['y_real','y_img']].to_numpy().tolist())

In [7]:
X_train = torch.view_as_complex(X_train).reshape(131520,1)
y_train = torch.view_as_complex(y_train).reshape(131520,1)

X_test = torch.view_as_complex(X_test).reshape(131520,1)
y_test = torch.view_as_complex(y_test).reshape(131520,1)

X_val  = torch.view_as_complex(X_val).reshape(131520,1)
y_val = torch.view_as_complex(y_val).reshape(131520,1)



In [8]:
y_train

tensor([[ 0.0012+0.0008j],
        [ 0.0025+0.0013j],
        [ 0.0032+0.0018j],
        ...,
        [-0.0032+0.0013j],
        [-0.0029-0.0001j],
        [-0.0032-0.0018j]])

In [12]:
# Define the neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(1, 30,dtype = torch.cfloat),
    nn.ReLU(),
    nn.Linear(30, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

# Define a loss function and an optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=4e-4)


# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: clamp is not supported for complex types

In [11]:
import torch
import torch.nn as nn

dummy_inputs = torch.rand((10, 15), dtype = torch.cfloat)

model = nn.Linear(15, 1, dtype = torch.cfloat)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

def criterion(x, y):
    dist = torch.sqrt((x**2+y**2))
    return torch.abs(torch.mean(dist))

optimizer.zero_grad()
outputs = model(dummy_inputs)
loss = criterion(outputs, torch.rand_like(outputs))
loss.backward()
optimizer.step()